In [84]:
import os
import pandas as pd
import numpy as np
import hvplot.pandas
from psaw import PushshiftAPI
from pathlib import Path
from newsapi import NewsApiClient
from dotenv import load_dotenv
from datetime import datetime
import datetime as dt
from dateutil.relativedelta import relativedelta, MO
import requests
import csv
from datetime import timezone
import time
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
load_dotenv()

True

In [85]:
api = PushshiftAPI()

In [86]:
# Download/Update the VADER Lexicon
nltk.download("vader_lexicon")

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dkv\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


**CARDANO - MONERO - CHAINLINK**
--


**SCRAPE CARDANO SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [106]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

cardano_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='cardano',
                            filter=['title', 'selftext'],
                            limit=None)


In [107]:
cardano_subm_df = pd.DataFrame([thing.d_ for thing in cardano_subm])

In [108]:
cardano_subm_df.tail()

,created,created_utc,selftext,title
89,1.591234e+09,1591219965,,Here Is Why Cardano (ADA) Is Pumping More Than...
90,1.591108e+09,1591093478,So far I have found that Etoro and other excha...,How do you buy Cardano (ADA) in the UK?
91,1.591097e+09,1591082369,,After a 20% week following Shelley imminent la...
92,1.591066e+09,1591051910,Cardano with the Shelley hype over-takes Tezos...,Some interesting Flappenings in the top 10
93,1.591054e+09,1591039869,,Cardano surpasses Tezos to become the 10th lar...


In [109]:
cardano_subm_df.drop(['created'], axis=1, inplace=True)

In [110]:
cardano_subm_df['created_utc'] = pd.to_datetime(cardano_subm_df['created_utc'],unit='s')

In [111]:
cardano_subm_df.set_index(['created_utc'], inplace=True)

In [112]:
cardano_subm_df.head()

,selftext,title
created_utc,,
2020-07-14 04:40:07,,On Governance (Part 1) New Charles Hoskinson t...
2020-07-14 04:38:57,,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:38:43,,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:37:47,,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:37:32,,Yo guys we can buy toothpaste with Cardano now...


In [113]:
cardano_subm_df.shape

(94, 2)

In [114]:
cardano_subm_df.reset_index(drop=True)

,selftext,title
0,,On Governance (Part 1) New Charles Hoskinson t...
1,,Yo guys we can buy toothpaste with Cardano now...
2,,Yo guys we can buy toothpaste with Cardano now...
3,,Yo guys we can buy toothpaste with Cardano now...
4,,Yo guys we can buy toothpaste with Cardano now...
5,,Cardano (ADA) is closing in on Bitcoin SV’s st...
6,\n\n[EXPERIMENT - Tracking Top 10 Cryptos of ...,"$1,000 invested in Top 10 Cryptos of 2019 now ..."
7,,Messari CEO: Cardano Has ‘Plenty of Room for G...
8,,Cardano founder warns against Youtube scams pr...
9,[removed],Is Cardano a money laundering scheme?


In [115]:
#fill in null values with a placeholder
cardano_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
cardano_subm_df['cardano_body'] = cardano_subm_df['title'] + ' ' + cardano_subm_df['selftext']
#subset dataframe of Cardano posts
cardano_subm_df = cardano_subm_df[['cardano_body']]

cardano_subm_df.head()

,cardano_body
created_utc,
2020-07-14 04:40:07,On Governance (Part 1) New Charles Hoskinson t...
2020-07-14 04:38:57,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:38:43,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:37:47,Yo guys we can buy toothpaste with Cardano now...
2020-07-14 04:37:32,Yo guys we can buy toothpaste with Cardano now...


**SCRAPE CARDANO COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [116]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

cardano_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='cardano',
                            filter=['body'],
                            limit=None)

In [117]:
cardano_comm_df = pd.DataFrame([thing.d_ for thing in cardano_comm])

In [118]:
cardano_comm_df.head()

,body,created,created_utc
0,Plenty of garbage projects have implemented po...,1.594764e+09,1594750019
1,Be fair and give it time. ETH has first mover ...,1.594762e+09,1594748091
2,so should i just sell all my ethereum lol i gu...,1.594757e+09,1594742286
3,How about Doge on Cardano to give it more func...,1.594755e+09,1594740773
4,"Lololololololol, Cardano will beat them to the...",1.594746e+09,1594731703


In [119]:
cardano_comm_df.drop(['created'], axis=1, inplace=True)

In [120]:
cardano_comm_df['created_utc'] = pd.to_datetime(cardano_comm_df['created_utc'],unit='s')

In [121]:
cardano_comm_df.set_index(['created_utc'], inplace=True)

In [122]:
cardano_comm_df.tail()

,body
created_utc,
2020-06-01 17:25:15,Cardano. Hello?
2020-06-01 13:54:23,Cardano
2020-06-01 10:26:41,"&gt; In Bitcoin, only a handful of mining pool..."
2020-06-01 10:16:46,"In Bitcoin, only a handful of mining pools con..."
2020-06-01 07:42:51,There will be a pullback. There was a pullback...


In [123]:
cardano_comm_df.shape

(737, 1)

In [124]:
cardano_comm_df = cardano_comm_df[['body']]
cardano_comm_df.columns = ['cardano_body']
cardano_comm_df.head()

,cardano_body
created_utc,
2020-07-14 18:06:59,Plenty of garbage projects have implemented po...
2020-07-14 17:34:51,Be fair and give it time. ETH has first mover ...
2020-07-14 15:58:06,so should i just sell all my ethereum lol i gu...
2020-07-14 15:32:53,How about Doge on Cardano to give it more func...
2020-07-14 13:01:43,"Lololololololol, Cardano will beat them to the..."


**COMBINE CARDANO DATAFRAMES**

In [125]:
#Combine Posts with Comments and reset index
cardano = pd.concat([cardano_subm_df,cardano_comm_df], axis =0)
cardano = cardano.reset_index(drop=True)
#Drop duplicate copies
cardano.drop_duplicates(keep='first',inplace=True)

cardano.head()

,cardano_body
0,On Governance (Part 1) New Charles Hoskinson t...
1,Yo guys we can buy toothpaste with Cardano now...
5,Cardano (ADA) is closing in on Bitcoin SV’s st...
6,"$1,000 invested in Top 10 Cryptos of 2019 now ..."
7,Messari CEO: Cardano Has ‘Plenty of Room for G...


**SCRAPE MONERO SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [126]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

xmr_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='Monero',
                            filter=['title', 'selftext'],
                            limit=None)

In [127]:
xmr_subm_df = pd.DataFrame([thing.d_ for thing in xmr_subm])

In [128]:
xmr_subm_df.tail()

,created,created_utc,selftext,title
37,1.591464e+09,1591450068,"Hi there reddit,I have a question to ask to y'...",Best multiwallet
38,1.591376e+09,1591361802,,Monero mining guide on Xmr-Stak-RX
39,1.591282e+09,1591267918,I’ve included basic information about Bitcoin ...,Bitcoin vs. Altcoins
40,1.591282e+09,1591267574,I’ve included basic information about Bitcoin ...,Bitcoin vs. Altcoins
41,1.591059e+09,1591044802,,Monero Interview: Shedding Light on the Coming...


In [129]:
xmr_subm_df.drop(['created'], axis=1, inplace=True)

In [130]:
xmr_subm_df['created_utc'] = pd.to_datetime(xmr_subm_df['created_utc'],unit='s')

In [131]:
xmr_subm_df.set_index('created_utc', inplace=True)

In [132]:
xmr_subm_df.head()

,selftext,title
created_utc,,
2020-07-10 01:31:52,Poloniex was the only exchange that I held mon...,Poloniex Lost 2019 Trade History
2020-07-08 13:00:39,[removed],What do you guys think about the future of Mon...
2020-07-08 06:34:37,I am trying to add support for cryptocurrency ...,How to creating QR codes? (supporting bitcoin ...
2020-07-07 20:25:11,,Andreas Antonopoulos on why Bitcoin likely won...
2020-07-07 15:38:43,A comment in the daily today reminded me of th...,How well would a Katy Perry portfolio have per...


In [133]:
xmr_subm_df.shape

(42, 2)

In [134]:
#fill in null values with a placeholder
xmr_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
xmr_subm_df['xmr_body'] = xmr_subm_df['title'] + ' ' + xmr_subm_df['selftext']
#subset dataframe of xmr posts
xmr_subm_df = xmr_subm_df[['xmr_body']]

xmr_subm_df.head()

,xmr_body
created_utc,
2020-07-10 01:31:52,Poloniex Lost 2019 Trade History Poloniex was ...
2020-07-08 13:00:39,What do you guys think about the future of Mon...
2020-07-08 06:34:37,How to creating QR codes? (supporting bitcoin ...
2020-07-07 20:25:11,Andreas Antonopoulos on why Bitcoin likely won...
2020-07-07 15:38:43,How well would a Katy Perry portfolio have per...


**SCRAPE MONERO COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [135]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

xmr_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='Monero',
                            filter=['body'],
                            limit=None)


In [136]:
xmr_comm_df = pd.DataFrame([thing.d_ for thing in xmr_comm])

In [137]:
xmr_comm_df.tail()

,body,created,created_utc
592,Monero will overtake ETH.,1.591039e+09,1591024550
593,Crap. Ethereum and Monero all the way.,1.591038e+09,1591023345
594,\*Monero,1.591035e+09,1591020687
595,There is also CPU mining which is better for s...,1.591034e+09,1591019255
596,"watch [Monero Means Money: Cryptocurrency 101,...",1.591014e+09,1590999585


In [138]:
xmr_comm_df.drop(['created'], axis=1, inplace=True)

In [139]:
xmr_comm_df['created_utc'] = pd.to_datetime(xmr_comm_df['created_utc'],unit='s')

In [140]:
xmr_comm_df.set_index('created_utc', inplace=True)

In [141]:
xmr_comm_df.head()

,body
created_utc,
2020-07-14 14:21:04,You could mine something with your gaming comp...
2020-07-14 03:49:49,"Monero, Maker and maybe Zcash, 0x, Kyber and C..."
2020-07-14 02:41:45,This is such a weird talking point. Most busin...
2020-07-13 13:19:25,Or we could use Monero.
2020-07-12 23:42:26,Might as well buy monero while you’re there an...


In [142]:
xmr_comm_df.shape

(597, 1)

In [143]:
xmr_comm_df = xmr_comm_df[['body']]
xmr_comm_df.columns = ['xmr_body']
xmr_comm_df.head()

,xmr_body
created_utc,
2020-07-14 14:21:04,You could mine something with your gaming comp...
2020-07-14 03:49:49,"Monero, Maker and maybe Zcash, 0x, Kyber and C..."
2020-07-14 02:41:45,This is such a weird talking point. Most busin...
2020-07-13 13:19:25,Or we could use Monero.
2020-07-12 23:42:26,Might as well buy monero while you’re there an...


**COMBINE MONERO DATAFRAMES**

In [144]:
#Combine Posts with Comments and reset index
xmr = pd.concat([xmr_subm_df,xmr_comm_df], axis =0)
xmr = xmr.reset_index(drop=True)
#Drop duplicate copies
xmr.drop_duplicates(keep='first',inplace=True)

xmr.head()

,xmr_body
0,Poloniex Lost 2019 Trade History Poloniex was ...
1,What do you guys think about the future of Mon...
2,How to creating QR codes? (supporting bitcoin ...
3,Andreas Antonopoulos on why Bitcoin likely won...
4,How well would a Katy Perry portfolio have per...


**SCRAPE CHAINLINK SUBMISSIONS IN CRYPTOCURRENCY SUBREDDIT**

In [87]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

link_subm = api.search_submissions(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='chainlink',
                            filter=['title', 'selftext'],
                            limit=None)


In [88]:
link_subm_df = pd.DataFrame([thing.d_ for thing in link_subm])

In [89]:
link_subm_df.tail()

,created,created_utc,selftext,title
62,1.591756e+09,1591742019,[removed],Chainlink vs EOS vs Tezos. Which has a better ...
63,1.591705e+09,1591690543,[removed],ARPA x Chainlink | Unlocking the Many Possibil...
64,1.591575e+09,1591560639,,Chainlink Joins InterWork | Coinbase Slashes U...
65,1.591063e+09,1591048686,,Chainlink price begins its typical breakout
66,1.591033e+09,1591018644,one of the most underrated coin. LTO The most ...,LTO Network!!


In [90]:
link_subm_df.drop(['created'], axis=1, inplace=True)

In [91]:
link_subm_df['created_utc'] = pd.to_datetime(link_subm_df['created_utc'],unit='s')

In [92]:
link_subm_df.set_index('created_utc', inplace=True)

In [93]:
link_subm_df.head()

,selftext,title
created_utc,,
2020-07-14 10:24:31,[removed],Is anyone holdin chainlink
2020-07-14 08:27:19,I want to start with this: I'm not a active us...,Chainlink- why do we hate it?
2020-07-14 06:56:29,,The Chainlink Misconception: More than Just a ...
2020-07-14 02:28:55,,"Is This The Next Chainlink Or Tezos 1,000% Coin?!"
2020-07-14 00:06:22,[removed],"Chainlink Token Price Surges, Briefly Breaks I..."


In [94]:
link_subm_df.shape

(67, 2)

In [95]:
#fill in null values with a placeholder
link_subm_df.fillna('.', inplace = True)
#create column 'body' that combines 'title' with 'selftext'
link_subm_df['link_body'] = link_subm_df['title'] + ' ' + link_subm_df['selftext']
#subset dataframe of chainlink posts
link_subm_df = link_subm_df[['link_body']]

link_subm_df.head()

,link_body
created_utc,
2020-07-14 10:24:31,Is anyone holdin chainlink [removed]
2020-07-14 08:27:19,Chainlink- why do we hate it? I want to start ...
2020-07-14 06:56:29,The Chainlink Misconception: More than Just a ...
2020-07-14 02:28:55,"Is This The Next Chainlink Or Tezos 1,000% Coi..."
2020-07-14 00:06:22,"Chainlink Token Price Surges, Briefly Breaks I..."


**SCRAPE CHAINLINK COMMENTS IN CRYPTOCURRENCY SUBREDDIT**

In [96]:
start_epoch=int(dt.datetime(2020, 6, 1).timestamp())

link_comm = api.search_comments(after=start_epoch,
                            subreddit='CryptoCurrency',
                            q='chainlink',
                            filter=['body'],
                            limit=None)

In [97]:
link_comm_df = pd.DataFrame([thing.d_ for thing in link_comm])

In [98]:
link_comm_df.tail()

,body,created,created_utc
447,Why get omisego when chainlink is light years ...,1.591084e+09,1591069500
448,Being a Chainlink holder is nice because they’...,1.591068e+09,1591053703
449,Please research Harmony (ONE). \n\n\n- It's th...,1.591064e+09,1591049732
450,Chainlink tries to bridge real world data so t...,1.591027e+09,1591012709
451,&gt; People investing in Chainlink now are foo...,1.591001e+09,1590986578


In [99]:
link_comm_df.drop(['created'], axis=1, inplace=True)

In [100]:
link_comm_df['created_utc'] = pd.to_datetime(link_comm_df['created_utc'],unit='s')

In [101]:
link_comm_df.set_index('created_utc', inplace=True)

In [102]:
link_comm_df.head()

,body
created_utc,
2020-07-14 17:43:08,When Chainlink pumps; I am awaken.
2020-07-14 17:32:10,"chainlink, but i heard it's a scam"
2020-07-14 17:20:09,"It is winding down, and yet I'm still buying t..."
2020-07-14 17:11:30,No it's not. Chainlink is an Ethereum token an...
2020-07-14 16:58:12,"Do not compare me xrp. Im speaking in facts, s..."


In [103]:
link_comm_df.shape

(452, 1)

In [104]:
link_comm_df = link_comm_df[['body']]
link_comm_df.columns = ['link_body']
link_comm_df.head()

,link_body
created_utc,
2020-07-14 17:43:08,When Chainlink pumps; I am awaken.
2020-07-14 17:32:10,"chainlink, but i heard it's a scam"
2020-07-14 17:20:09,"It is winding down, and yet I'm still buying t..."
2020-07-14 17:11:30,No it's not. Chainlink is an Ethereum token an...
2020-07-14 16:58:12,"Do not compare me xrp. Im speaking in facts, s..."


**COMBINE CHAINLINK DATAFRAMES**

In [105]:
#Combine Posts with Comments and reset index
link = pd.concat([link_subm_df,link_comm_df], axis =0)
link = link.reset_index(drop=True)
#Drop duplicate copies
link.drop_duplicates(keep='first',inplace=True)

link.head()

,link_body
0,Is anyone holdin chainlink [removed]
1,Chainlink- why do we hate it? I want to start ...
2,The Chainlink Misconception: More than Just a ...
3,"Is This The Next Chainlink Or Tezos 1,000% Coi..."
4,"Chainlink Token Price Surges, Briefly Breaks I..."


**COMBINE MONERO, CARDANO & CHAINLINK DF'S**

In [145]:
#create dataframe of two columns: xmr_body & cardano_body
crypto_cc_df = pd.concat([xmr,cardano,link],axis=1)
crypto_cc_df.dropna()
crypto_cc_df.reset_index(drop=True,inplace=True)

crypto_cc_df.head()

,xmr_body,cardano_body,link_body
0,Poloniex Lost 2019 Trade History Poloniex was ...,On Governance (Part 1) New Charles Hoskinson t...,Is anyone holdin chainlink [removed]
1,What do you guys think about the future of Mon...,Yo guys we can buy toothpaste with Cardano now...,Chainlink- why do we hate it? I want to start ...
2,How to creating QR codes? (supporting bitcoin ...,NaN,The Chainlink Misconception: More than Just a ...
3,Andreas Antonopoulos on why Bitcoin likely won...,NaN,"Is This The Next Chainlink Or Tezos 1,000% Coi..."
4,How well would a Katy Perry portfolio have per...,NaN,"Chainlink Token Price Surges, Briefly Breaks I..."


**ANALYZE SENTIMENT**

In [146]:
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [147]:
xmr_sent = {
    "xmr_compound": [],
    "xmr_pos": [],
    "xmr_neu": [],
    "xmr_neg": [],
    "xmr_sent": [],
}
cardano_sent = {
    "cardano_compound": [],
    "cardano_pos": [],
    "cardano_neu": [],
    "cardano_neg": [],
    "cardano_sent": [],
}
link_sent = {
    "link_compound": [],
    "link_pos": [],
    "link_neu": [],
    "link_neg": [],
    "link_sent": [],
}

# Get sentiment for the text and the title
for index, row in crypto_cc_df.iterrows():
    try:
        # Sentiment scoring with VADER
        xmr_sentiment = analyzer.polarity_scores(row["xmr_body"])
        xmr_sent["xmr_compound"].append(xmr_sentiment["compound"])
        xmr_sent["xmr_pos"].append(xmr_sentiment["pos"])
        xmr_sent["xmr_neu"].append(xmr_sentiment["neu"])
        xmr_sent["xmr_neg"].append(xmr_sentiment["neg"])
        xmr_sent["xmr_sent"].append(get_sentiment(xmr_sentiment["compound"]))

        cardano_sentiment = analyzer.polarity_scores(row["cardano_body"])
        cardano_sent["cardano_compound"].append(cardano_sentiment["compound"])
        cardano_sent["cardano_pos"].append(cardano_sentiment["pos"])
        cardano_sent["cardano_neu"].append(cardano_sentiment["neu"])
        cardano_sent["cardano_neg"].append(cardano_sentiment["neg"])
        cardano_sent["cardano_sent"].append(get_sentiment(cardano_sentiment["compound"]))
        
        link_sentiment = analyzer.polarity_scores(row["link_body"])
        link_sent["link_compound"].append(link_sentiment["compound"])
        link_sent["link_pos"].append(link_sentiment["pos"])
        link_sent["link_neu"].append(link_sentiment["neu"])
        link_sent["link_neg"].append(link_sentiment["neg"])
        link_sent["link_sent"].append(get_sentiment(link_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
xmr_sentiment = pd.DataFrame(xmr_sent)
cardano_sentiment = pd.DataFrame(cardano_sent)
link_sentiment = pd.DataFrame(link_sent)
crypto_df = crypto_cc_df.join(xmr_sentiment).join(cardano_sentiment).join(link_sentiment)

crypto_df.head()

,xmr_body,cardano_body,link_body,xmr_compound,xmr_pos,xmr_neu,xmr_neg,xmr_sent,cardano_compound,cardano_pos,cardano_neu,cardano_neg,cardano_sent,link_compound,link_pos,link_neu,link_neg,link_sent
0,Poloniex Lost 2019 Trade History Poloniex was ...,On Governance (Part 1) New Charles Hoskinson t...,Is anyone holdin chainlink [removed],0.4515,0.089,0.816,0.095,1.0,0.0000,0.000,1.000,0.000,0.0,0.0000,0.000,1.000,0.000,0.0
1,What do you guys think about the future of Mon...,Yo guys we can buy toothpaste with Cardano now...,Chainlink- why do we hate it? I want to start ...,0.0000,0.000,1.000,0.000,0.0,0.6360,0.259,0.741,0.000,1.0,0.4071,0.142,0.735,0.123,1.0
2,How to creating QR codes? (supporting bitcoin ...,NaN,The Chainlink Misconception: More than Just a ...,0.8762,0.138,0.862,0.000,1.0,0.0000,0.000,1.000,0.000,0.0,0.3182,0.365,0.635,0.000,1.0
3,Andreas Antonopoulos on why Bitcoin likely won...,NaN,"Is This The Next Chainlink Or Tezos 1,000% Coi...",0.0000,0.000,1.000,0.000,0.0,0.9991,0.125,0.843,0.032,1.0,-0.7975,0.058,0.865,0.076,-1.0
4,How well would a Katy Perry portfolio have per...,NaN,"Chainlink Token Price Surges, Briefly Breaks I...",0.8847,0.074,0.905,0.020,1.0,0.0000,0.000,1.000,0.000,0.0,0.0000,0.000,1.000,0.000,0.0


In [148]:
crypto_df.hvplot.step(y=['xmr_sent', 'cardano_sent', 'link_sent'],width=3000, height=300, xlim=(0, 618), legend=('left'))

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [149]:
crypto_df.describe()

,xmr_compound,xmr_pos,xmr_neu,xmr_neg,xmr_sent,cardano_compound,cardano_pos,cardano_neu,cardano_neg,cardano_sent,link_compound,link_pos,link_neu,link_neg,link_sent
count,621.000000,621.000000,621.000000,621.000000,621.000000,615.000000,615.000000,615.000000,615.000000,615.000000,494.000000,494.000000,494.000000,494.000000,494.000000
mean,0.236726,0.110738,0.831084,0.058184,0.346216,0.296217,0.114572,0.833197,0.052221,0.429268,0.137872,0.096184,0.830200,0.073640,0.226721
std,0.535963,0.099793,0.114186,0.077288,0.831403,0.506305,0.106802,0.121761,0.078329,0.805795,0.530329,0.100426,0.128196,0.104783,0.862337
min,-0.998900,0.000000,0.393000,0.000000,-1.000000,-0.953800,0.000000,0.323000,0.000000,-1.000000,-0.993300,0.000000,0.394000,0.000000,-1.000000
25%,0.000000,0.000000,0.760000,0.000000,0.000000,0.000000,0.011500,0.774000,0.000000,0.000000,-0.203575,0.000000,0.761250,0.000000,-1.000000
50%,0.273200,0.104000,0.830000,0.033000,1.000000,0.391900,0.102000,0.841000,0.023000,1.000000,0.083600,0.083000,0.831500,0.032500,1.000000
75%,0.710900,0.164000,0.908000,0.092000,1.000000,0.729500,0.164000,0.909000,0.077500,1.000000,0.571650,0.150750,0.923000,0.106000,1.000000
max,0.998700,0.537000,1.000000,0.421000,1.000000,0.999900,0.677000,1.000000,0.500000,1.000000,0.998300,0.494000,1.000000,0.606000,1.000000
